I used the code of [GISLR: PyTorch->TFLite Baseline](https://www.kaggle.com/code/myso1987/gislr-pytorch-tflite-baseline/notebook) as a baseline.
If you want to add Arcface layer just change `CFG.arcface = True`.

## Version1
* Added  ArcFace loss with the cross entropy loss
* Train_test_split for validation
* `CV:0.6458`  , `LB:0.45`

## Version2
* No Arcface
* Train_test_split for validation
* `CV:0.6119`  , `LB:0.39`

## Version3
* Added  ArcFace loss with the cross entropy loss
* StratifiedGroupKFold for validation
* `epochs=350`
* `CV:0.39`  , `LB:0.42`

## Version4
* Added  ArcFace loss with the cross entropy loss
* StratifiedGroupKFold for validation
* Great feature generation using PyTorch model based on this [notebook](https://www.kaggle.com/code/mayukh18/end-to-end-pytorch-training-submission)
* `epochs=400`, `T_max=5`, `batch_size=256`
* `CV:0.52`  , `LB:0.19`

## Version5
* No ArcFace
* StratifiedGroupKFold for validation
* Great feature generation using PyTorch model based on this [notebook](https://www.kaggle.com/code/mayukh18/end-to-end-pytorch-training-submission)
* `epochs=300`, `T_max=10`, `batch_size=128`
* `CV:0.51`  , `LB:0.19`

## Version6
* No ArcFace
* Model from [🤟 GISLR 🤟 - 📚Learn – 🔭EDA – 🤖Baseline](https://www.kaggle.com/code/dschettler8845/gislr-learn-eda-baseline)
* StratifiedGroupKFold for validation
* Great feature generation using PyTorch model based on this [notebook](https://www.kaggle.com/code/mayukh18/end-to-end-pytorch-training-submission)
* `epochs=300`, `T_max=10`, `batch_size=128`
* `CV:0.48`  , `LB:0.50`

## Version7
* No ArcFace
* Model from [🤟 GISLR 🤟 - 📚Learn – 🔭EDA – 🤖Baseline](https://www.kaggle.com/code/dschettler8845/gislr-learn-eda-baseline)
* StratifiedGroupKFold for validation
* Great feature generation using PyTorch model based on this [notebook](https://www.kaggle.com/code/mayukh18/end-to-end-pytorch-training-submission) and my [notebook](https://www.kaggle.com/code/medali1992/isolated-sign-language-aggregation-preparation) for feature generation.
* Removed the 'z' axis and face features and rerplaced them with lips as stated [here.](https://www.kaggle.com/competitions/asl-signs/discussion/391812)
* `epochs=500`, `T_max=10`, `batch_size=512`, `num_blocks=3`

# Pip Install Modules 

In [ ]:
!pip install onnx_tf
!pip install tflite-runtime
!pip install -q --upgrade wandb

In [ ]:
# Install nb_black for autoformatting
!pip install nb_black --quiet
%load_ext lab_black

# Libraries

In [ ]:
import numpy as np
import pandas as pd
import math
import random
import time
from collections import OrderedDict
import tensorflow as tf
from tqdm import tqdm
import json
import os
import gc
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GroupKFold, StratifiedGroupKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.optim.optimizer import Optimizer
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import (
    CosineAnnealingWarmRestarts,
    CosineAnnealingLR,
    ReduceLROnPlateau,
)
from torchinfo import summary

import onnx
import onnx_tf
from onnx_tf.backend import prepare

import warnings

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VERSION = 7

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = accuracy_score(y_true, y_pred)
    return score


def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def load_relevant_data_subset_with_imputation(pq_path):
    data_columns = ["x", "y", "z"]
    data = pd.read_parquet(pq_path, columns=data_columns)
    data.replace(np.nan, 0, inplace=True)
    n_frames = int(len(data) / CFG.rows_per_frame)
    data = data.values.reshape(n_frames, CFG.rows_per_frame, len(data_columns))
    return data.astype(np.float32)


def load_relevant_data_subset(pq_path):
    data_columns = ["x", "y"]
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / CFG.rows_per_frame)
    data = data.values.reshape(n_frames, CFG.rows_per_frame, len(data_columns))
    return data.astype(np.float32)


def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic

# Configuration

In [ ]:
class CFG:
    num_workers = 2
    apex = False
    scheduler = "CosineAnnealingLR"  # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts','OneCycleLR']
    epochs = 500
    print_freq = 200
    # CosineAnnealingLR params
    cosanneal_params = {"T_max": 5, "eta_min": 3 * 1e-5, "last_epoch": -1}
    # ReduceLROnPlateau params
    reduce_params = {
        "mode": "min",
        "factor": 0.1,
        "patience": 6,
        "eps": 1e-6,
        "verbose": True,
    }
    # CosineAnnealingWarmRestarts params
    cosanneal_res_params = {"T_0": 3, "eta_min": 1e-6, "T_mult": 1, "last_epoch": -1}
    # OneCycleLR params
    onecycle_params = {
        "pct_start": 0.1,
        "div_factor": 1e1,
        "max_lr": 1e-3,
        "steps_per_epoch": 3,
        "epochs": 3,
    }
    momentum = 0.9
    model_name = "NN_ArcFace"
    lr = 3 * 1e-4
    weight_decay = 1e-4
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    data_path = "../input/asl-signs/"
    debug = False
    arcface = False
    use_aggregation_dataset = True
    target_size = 250
    rows_per_frame = 543
    batch_size = 512
    train = True
    early_stop = True
    target_col = "label"
    scale = 30.0
    margin = 0.50
    easy_margin = False
    ls_eps = 0.0
    fc_dim = 512
    early_stopping_steps = 5
    grad_cam = False
    seed = 42

# Directory Settings

In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = f"./{CFG.model_name}_version{VERSION}/"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)


def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = init_logger()

# Load Data

In [ ]:
train = pd.read_csv(f"{CFG.data_path}train.csv")
label_index = read_dict(f"{CFG.data_path}sign_to_prediction_index_map.json")
index_label = dict([(label_index[key], key) for key in label_index])
train["label"] = train["sign"].map(lambda sign: label_index[sign])

if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=4000, random_state=CFG.seed).reset_index(drop=True)

train.head()

# Data Preparation

## Source: [End-to-End Pytorch Training + Submission](https://www.kaggle.com/code/mayukh18/end-to-end-pytorch-training-submission)

In [ ]:
# https://www.kaggle.com/competitions/asl-signs/discussion/391812#2168354
lipsUpperOuter = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter = [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner = [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
lips = lipsUpperOuter + lipsLowerOuter + lipsUpperInner + lipsLowerInner


class FeatureGen(nn.Module):
    def __init__(self):
        super(FeatureGen, self).__init__()
        pass

    def forward(self, x):
        x = x[:, :, :2]
        lips_x = x[:, lips, :].contiguous().view(-1, 43 * 2)
        lefth_x = x[:, 468:489, :].contiguous().view(-1, 21 * 2)
        pose_x = x[:, 489:522, :].contiguous().view(-1, 33 * 2)
        righth_x = x[:, 522:, :].contiguous().view(-1, 21 * 2)

        lefth_x = lefth_x[~torch.any(torch.isnan(lefth_x), dim=1), :]
        righth_x = righth_x[~torch.any(torch.isnan(righth_x), dim=1), :]

        x1m = torch.mean(lips_x, 0)
        x2m = torch.mean(lefth_x, 0)
        x3m = torch.mean(pose_x, 0)
        x4m = torch.mean(righth_x, 0)

        x1s = torch.std(lips_x, 0)
        x2s = torch.std(lefth_x, 0)
        x3s = torch.std(pose_x, 0)
        x4s = torch.std(righth_x, 0)

        xfeat = torch.cat([x1m, x2m, x3m, x4m, x1s, x2s, x3s, x4s], axis=0)
        xfeat = torch.where(
            torch.isnan(xfeat), torch.tensor(0.0, dtype=torch.float32), xfeat
        )

        return xfeat


feature_converter = FeatureGen()
X = np.load(
    "/kaggle/input/isolated-sign-language-aggregation-preparation/feature_data.npy"
)
y = np.load(
    "/kaggle/input/isolated-sign-language-aggregation-preparation/feature_labels.npy"
)
print(X.shape, y.shape)

# Model Tracking

In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_key")

import wandb

wandb.login(key=wandb_api)


def class2dict(f):
    return dict(
        (name, getattr(f, name)) for name in dir(f) if not name.startswith("__")
    )


run = wandb.init(
    project="GISLR Competition",
    name=f"{CFG.model_name}_Version{VERSION}",
    config=class2dict(CFG),
    group=CFG.model_name,
    job_type="train",
)

# DataSet

In [ ]:
class Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.X[i].astype(np.float32), self.y[i]

# Model

In [ ]:
class ArcMarginProduct(nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        scale=30.0,
        margin=0.50,
        easy_margin=False,
        ls_eps=0.0,
    ):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output


class Model(nn.Module):
    def __init__(self, cfg):
        super(Model, self).__init__()
        self.cfg = cfg
        self.lin_bn_mish = nn.Sequential(
            OrderedDict(
                [
                    ("lin_mish1", lin_bn_mish(472, 512)),
                    ("lin_mish2", lin_bn_mish(512, 256)),
                    ("lin_mish3", lin_bn_mish(256, 256)),
                    ("lin_mish4", lin_bn_mish(256, 128)),
                ]
            )
        )

        self.final = ArcMarginProduct(
            128,
            self.cfg.target_size,
            scale=self.cfg.scale,
            margin=self.cfg.margin,
            easy_margin=False,
            ls_eps=0.0,
        )
        self.fc_probs = nn.Linear(128, self.cfg.target_size)

    def forward(self, x, label):
        feature = self.lin_bn_mish(x)
        if self.cfg.arcface:
            arcface = self.final(feature, label)
            probs = self.fc_probs(feature)
            return probs, arcface
        else:
            probs = self.fc_probs(feature)
            return probs


def lin_bn_mish(input_dim, output_dim):
    return nn.Sequential(
        OrderedDict(
            [
                ("lin", nn.Linear(input_dim, output_dim, bias=False)),
                ("bn", nn.BatchNorm1d(output_dim)),
                ("dropout", nn.Dropout(0.2)),
                ("relu", nn.Mish()),
            ]
        )
    )

In [ ]:
class ASLLinearModel(torch.nn.Module):
    def __init__(
        self,
        in_features: int,
        first_out_features: int,
        num_classes: int,
        num_blocks: int,
        drop_rate: float,
    ):
        super(ASLLinearModel, self).__init__()

        blocks = []
        out_features = first_out_features
        for idx in range(num_blocks):
            if idx == num_blocks - 1:
                out_features = num_classes

            blocks.append(self._make_block(in_features, out_features, drop_rate))

            in_features = out_features
            out_features = out_features // 2

        self.model = nn.Sequential(*blocks)
        print(self.model)

    def _make_block(self, in_features, out_features, drop_rate):
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            nn.ReLU(),
            nn.Dropout(drop_rate),
        )

    def forward(self, x):
        return self.model(x)

# Helper Function

In [ ]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    if CFG.apex:
        scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (features, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        features = features.to(device).float()
        labels = labels.to(device).long()
        batch_size = labels.size(0)
        if CFG.apex:
            with autocast():
                if CFG.arcface:
                    probs, arcface = model(features, labels)
                    arcface_loss = nn.CrossEntropyLoss()(arcface, labels)
                    loss = criterion(probs, labels)
                else:
                    y_preds = model(features, labels)
                    loss = criterion(y_preds, labels)
        else:
            if CFG.arcface:
                probs, arcface = model(features, labels)
                arcface_loss = nn.CrossEntropyLoss()(arcface, labels)
                loss = criterion(probs, labels)
            else:
                y_preds = model(features)
                loss = criterion(y_preds, labels)
        # record loss
        if CFG.arcface:
            loss = 0.5 * loss + 0.5 * arcface_loss
            losses.update(loss.item(), batch_size)
        else:
            losses.update(loss.item(), batch_size)

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), CFG.max_grad_norm
        )
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            if CFG.apex:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader) - 1):
            print(
                "Epoch: [{0}][{1}/{2}] "
                "Elapsed {remain:s} "
                "Loss: {loss.val:.4f}({loss.avg:.4f}) "
                "Grad: {grad_norm:.4f} "
                "LR: {lr:.6f}  ".format(
                    epoch + 1,
                    step,
                    len(train_loader),
                    remain=timeSince(start, float(step + 1) / len(train_loader)),
                    loss=losses,
                    grad_norm=grad_norm,
                    lr=scheduler.get_lr()[0],
                )
            )
        wandb.log(
            {
                f"loss": losses.val,
                f"lr": scheduler.get_lr()[0],
            }
        )
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (features, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        features = features.to(device).float()
        labels = labels.to(device).long()
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            if CFG.arcface:
                y_preds, _ = model(features, labels)
            else:
                y_preds = model(features)

        preds.append(y_preds.softmax(1).to("cpu").numpy())
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader) - 1):
            print(
                "EVAL: [{0}/{1}] "
                "Elapsed {remain:s} "
                "Loss: {loss.val:.4f}({loss.avg:.4f}) ".format(
                    step,
                    len(valid_loader),
                    loss=losses,
                    remain=timeSince(start, float(step + 1) / len(valid_loader)),
                )
            )
    predictions = np.concatenate(preds)
    return losses.avg, predictions

# Train Loop

In [ ]:
# Seed for producing results
seed_torch(seed=45)

# ====================================================
# loader
# ====================================================
groups = train["path"].map(lambda x: x.split("/")[1])
sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True)
for i, (train_index, valid_index) in enumerate(sgkf.split(X, y, groups)):
    train_index = train_index
    valid_index = valid_index
    print(f"Fold {i}:")
    print(f"  Train index shape: {train_index.shape}")
    print(f"         group={groups[train_index]}")
    print(f"  Valid index shape:  {valid_index.shape}")
    print(f"         group={groups[valid_index]}")
    break
X_train, X_val, y_train, y_val = (
    X[train_index],
    X[valid_index],
    y[train_index],
    y[valid_index],
)
train_dataset = Dataset(X_train, y_train)
valid_dataset = Dataset(X_val, y_val)


train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.batch_size,
    shuffle=True,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=True,
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=CFG.batch_size,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False,
)


# ====================================================
# scheduler
# ====================================================
def get_scheduler(optimizer):
    if CFG.scheduler == "ReduceLROnPlateau":
        scheduler = ReduceLROnPlateau(optimizer, **CFG.reduce_params)
    elif CFG.scheduler == "CosineAnnealingLR":
        scheduler = CosineAnnealingLR(optimizer, **CFG.cosanneal_params)
    elif CFG.scheduler == "CosineAnnealingWarmRestarts":
        scheduler = CosineAnnealingWarmRestarts(optimizer, **CFG.reduce_params)
    return scheduler


# ====================================================
# model & optimizer
# ====================================================
model = ASLLinearModel(
    in_features=472,
    first_out_features=1024,
    num_classes=250,
    num_blocks=3,
    drop_rate=0.4,
)
model.to(device)

optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
scheduler = get_scheduler(optimizer)

# ====================================================
# loop
# ====================================================
criterion = nn.CrossEntropyLoss()
best_score = 0
for epoch in range(CFG.epochs):
    start_time = time.time()

    # train
    avg_loss = train_fn(
        train_loader, model, criterion, optimizer, epoch, scheduler, device
    )

    # eval
    avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)

    if isinstance(scheduler, ReduceLROnPlateau):
        scheduler.step(avg_val_loss)
    elif isinstance(scheduler, CosineAnnealingLR):
        scheduler.step()
    elif isinstance(scheduler, CosineAnnealingWarmRestarts):
        scheduler.step()

    score = get_score(y_val, preds.argmax(1))

    elapsed = time.time() - start_time

    LOGGER.info(
        f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
    )
    LOGGER.info(f"Epoch {epoch+1} - Score: {score:.4f}")
    wandb.log(
        {
            f"epoch": epoch + 1,
            f"avg_train_loss": avg_loss,
            f"avg_val_loss": avg_val_loss,
            f"score": score,
        }
    )

    if best_score < score:
        best_score = score
        LOGGER.info(f"Epoch {epoch+1} - Save Best score: {best_score:.4f} Model")
        torch.save(
            model.state_dict(),
            OUTPUT_DIR + f"{CFG.model_name}_best_score_version{VERSION}.pth",
        )
LOGGER.info(f"Our CV score is {best_score}")

# Tensorflow Conversion

## Source: [End-to-End Pytorch Training + Submission](https://www.kaggle.com/code/mayukh18/end-to-end-pytorch-training-submission)

In [ ]:
sample_input = torch.rand((50, 543, 2))
onnx_feat_gen_path = "feature_gen.onnx"

feature_converter.eval()

torch.onnx.export(
    feature_converter,  # PyTorch Model
    sample_input,  # Input tensor
    onnx_feat_gen_path,  # Output file (eg. 'output_model.onnx')
    opset_version=12,  # Operator support version
    input_names=["input"],  # Input tensor name (arbitary)
    output_names=["output"],  # Output tensor name (arbitary)
    dynamic_axes={"input": {0: "input"}},
)

In [ ]:
model_infe = ASLLinearModel(
    in_features=472,
    first_out_features=1024,
    num_classes=250,
    num_blocks=3,
    drop_rate=0.4,
)

model_infe.load_state_dict(
    torch.load(OUTPUT_DIR + f"{CFG.model_name}_best_score_version{VERSION}.pth"),
    strict=False,
)
model_infe = model_infe.to(device)

In [ ]:
sample_input = torch.rand((1, 472)).to(device)
onnx_model_path = "asl_model.onnx"

model_infe.eval()

torch.onnx.export(
    model_infe,  # PyTorch Model
    sample_input,  # Input tensor
    onnx_model_path,  # Output file (eg. 'output_model.onnx')
    opset_version=12,  # Operator support version
    input_names=["input"],  # Input tensor name (arbitary)
    output_names=["output"],  # Output tensor name (arbitary)
    dynamic_axes={"input": {0: "input"}},
)

In [ ]:
import onnx
from onnx_tf.backend import prepare


tf_feat_gen_path = "/kaggle/working/tf_feat_gen"
onnx_feat_gen = onnx.load(onnx_feat_gen_path)
tf_rep = prepare(onnx_feat_gen)
tf_rep.export_graph(tf_feat_gen_path)


tf_model_path = "/kaggle/working/tf_model"
onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

# Final Inference Model in Tensorflow
Both of the converted models will be used here one after another.

In [ ]:
import tensorflow as tf


class ASLInferModel(tf.Module):
    def __init__(self):
        super(ASLInferModel, self).__init__()
        self.feature_gen = tf.saved_model.load(tf_feat_gen_path)
        self.model = tf.saved_model.load(tf_model_path)
        self.feature_gen.trainable = False
        self.model.trainable = False

    @tf.function(
        input_signature=[
            tf.TensorSpec(shape=[None, 543, 2], dtype=tf.float32, name="inputs")
        ]
    )
    def call(self, input):
        output_tensors = {}
        features = self.feature_gen(**{"input": input})["output"]
        output_tensors["outputs"] = self.model(
            **{"input": tf.expand_dims(features, 0)}
        )["output"][0, :]
        return output_tensors


mytfmodel = ASLInferModel()
tf.saved_model.save(
    mytfmodel,
    "/kaggle/working/tf_infer_model",
    signatures={"serving_default": mytfmodel.call},
)

# Submission

In [ ]:
# Convert the model

tf_infer_model_path = "/kaggle/working/tf_infer_model"
converter = tf.lite.TFLiteConverter.from_saved_model(tf_infer_model_path)
tflite_model = converter.convert()

tflite_model_path = "model.tflite"

# Save the model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

In [ ]:
ROWS_PER_FRAME = 543  # number of landmarks per frame
pq_path = "/kaggle/input/asl-signs/train_landmark_files/53618/1001379621.parquet"

import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter(tflite_model_path)
interpreter.allocate_tensors()

found_signatures = list(interpreter.get_signature_list().keys())

# if REQUIRED_SIGNATURE not in found_signatures:
#     raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=load_relevant_data_subset(pq_path))
sign = np.argmax(output["outputs"])

print(sign, output["outputs"].shape)

In [ ]:
!zip submission.zip $tflite_model_path